# DLA-34 in Keras with HSwish as option

In [7]:
from keras.layers import Conv2D, BatchNormalization, Input, MaxPool2D
from keras.activations import relu
from keras.models import Sequential, Model
from keras.layers.merge import add, concatenate
from keras.optimizers import SGD

from keras import backend as K
from keras.layers import Layer
    
import numpy as np

In [2]:
class ReLU(Layer):
    def __init__(self, **kwargs):
        super(ReLU, self).__init__(**kwargs)

    def build(self, input_shape):
        super(ReLU, self).build(input_shape)

    def call(self, x):
        return K.relu(x)

    def compute_output_shape(self, input_shape):
        return input_shape
    
class HSwish(Layer):
    def __init__(self, **kwargs):
        super(HSwish, self).__init__(**kwargs)

    def build(self, input_shape):
        super(HSwish, self).build(input_shape)

    def call(self, x):
        six = K.ones_like(x)*6
        return x* K.minimum(K.relu(x+3),six)/6

    def compute_output_shape(self, input_shape):
        return input_shape

In [3]:
class Tree():
    def __init__(self, name):
        self.root = None
        self.tree1 = None
        self.tree2 = None
        self.level_root = None
        self.root_dim = None
        self.downsample = None
        self.project = None
        self.levels = None
        
        self.name = name
        
    def __print_indent(self, indent, text, end='\n'):
        print( "".join([' ']*indent), end='')
        print(text, end = end)
              
    def print(self, indent = 0):
        self.__print_indent(1 if indent>0 else 0, "[NODE: "+str(self.name), end=' ')
        self.__print_indent(0, "lr:"+str(self.level_root), end=' ')
        self.__print_indent(0, "rdim:"+str(self.root_dim), end=' ')
        self.__print_indent(0, "downs: "+str(self.downsample==None), end=' ')
        self.__print_indent(0, "proj: "+str(self.project==None), end=' ')
        self.__print_indent(0, "levs: "+str(self.levels), end=']\n')
        
        if self.root is None:
              self.__print_indent(indent+2, "R1> None")
        else:
              self.__print_indent(indent+2, "R1> conv,bn,act ")
              
        if isinstance(self.tree1,Tree):
            self.__print_indent(indent+2, "T1>", end='')
            self.tree1.print(indent=indent+4)
        elif self.tree1 is not None:
            self.__print_indent(indent+2, "T1> conv,bn")

        if isinstance(self.tree2,Tree):
            self.__print_indent(indent+2, "T2>", end='')
            self.tree2.print(indent=indent+4)
        elif self.tree2 is not None:
            self.__print_indent(indent+2, "T2> conv,bn")

In [4]:
class DlaKeras():
    def __init__(self, input_shape = (512,512,3),  
                 levels=[1, 1, 1, 2, 2, 1],
                 planes=[16, 32, 64, 128, 256, 512],
                 data_format = "NHWC", activation_function="relu", debug=False):
        self.norm_axis = 1 if data_format =="NCHW" else -1
        self.input = Input(shape = input_shape)
        self.activation_function = activation_function
        
        self.base_layer = self._make_simple_block(planes[0], kernel_size=7)
        self.level0 = self._make_conv_level(planes[0], levels[0], stride=1)
        self.level1 = self._make_conv_level(planes[1], levels[1], stride=2)
        
        self.level2 = self._make_tree("L2_", levels[2], planes[1], planes[2], level_root=False)
        self.level3 = self._make_tree("L3_",levels[3], planes[2], planes[3])
        self.level4 = self._make_tree("L4_",levels[4], planes[3], planes[4])
        self.level5 = self._make_tree("L5_",levels[5], planes[4], planes[5])
        
        self.debug = debug
        if debug:
            self.level2.print()
            self.level3.print()
            self.level4.print()
            self.level5.print()
        
    def _make_tree(self, name, levels, in_planes, planes, stride=2, level_root=True, root_dim=0):
        result = Tree(name)
        
        if root_dim == 0:
            root_dim = 2 * planes
        if level_root:
            root_dim += in_planes
        
        if levels==1:
            result.tree1 = self._make_basic_block(planes, stride)
            result.tree2 = self._make_basic_block(planes, 1)
        else:
            result.tree1 = self._make_tree(name+"1",levels-1, in_planes, planes, stride=stride)
            result.tree2 = self._make_tree(name+"2",levels-1, planes, planes, stride=1, root_dim=root_dim + planes)
            
        if levels == 1:
            result.root = self._make_root(root_dim, planes)   # TODO minor: remove root_dim ?
        
        result.level_root = level_root
        result.root_dim = root_dim
        result.levels = levels
        if stride > 1:
            result.downsample = MaxPool2D(pool_size=(stride, stride), strides=(stride, stride), padding = 'same')
            
        if in_planes != planes:
            result.project = []
            result.project.append(Conv2D(filters=planes, kernel_size=1, padding='same', strides=(1,1), use_bias=False))
            result.project.append(BatchNormalization(axis=self.norm_axis))    

        return result

    def _make_root(self, in_planes, planes):
        return self._make_simple_block(planes)
    
    def _make_basic_block(self, planes, stride=1):
        layers=[]
        
        conv1 = Conv2D(filters=planes, kernel_size=3, padding='same', strides=(stride,stride), use_bias=False)
        bn1 = BatchNormalization(axis=self.norm_axis)
        if self.activation_function == "hswish":
            activation = HSwish()
        else:
            activation = ReLU()
        conv2 = Conv2D(filters=planes, kernel_size=3, padding='same', strides=(1,1), use_bias=False)
        bn2 = BatchNormalization(axis=self.norm_axis)
        
        layers = [conv1,bn1,activation, conv2, bn2]
        return layers
    
    def _make_simple_block(self, planes, kernel_size=3, stride = 1):
        
        conv = Conv2D(filters=planes, kernel_size=kernel_size, padding='same', strides=(stride,stride), use_bias=False)
        bn = BatchNormalization(axis=self.norm_axis)
        if self.activation_function == "hswish":
            activation = HSwish()
        else:
            activation = ReLU()
        
        layers = [conv,bn,activation]
        
        return layers
    
    def _make_conv_level(self, planes, levels, stride = 1):
        layers = []
        for i in range(levels):
            layers.extend(self._make_simple_block(planes, stride = stride))
        return layers
            
    def _build_list(self, x, layers):
        for l in layers:
            x = l(x)
        return x
    
    def _build_root(self, root, *x):
        x = list(x)
        conv = root[0]
        bn = root[1]
        act = root[2]
        
        #print("--", len(x))
        #for i in x:
            #print(i.shape)
        #print("---")
        
        x = concatenate(x, axis = self.norm_axis)
        #print("Concatenated: ",x.shape)
        x = conv(x)
        x = bn(x)
        x = act(x)

        return x
    
    def _build_tree(self,x, tree, children=None, residual=None):
        if not isinstance(tree, Tree):
            return self._build_list(x,tree)
        
        #print("\nCALL:"+tree.name)
        #print("input shape:", x.shape)
        
        children = [] if children is None else children
        #print("children", len(children))
        #print("bottom before", x.shape)
        bottom = tree.downsample(x) if tree.downsample else x
        #print("bottom after", bottom.shape)
        
        #print("residual before", bottom.shape)
        residual = self._build_list(bottom,tree.project) if tree.project else bottom
        #print("residual after", residual.shape)
        
        if tree.level_root:
            children.append(bottom)

        x1 = self._build_tree(x, tree.tree1, residual=residual)

        if tree.levels == 1:
            x2 = self._build_tree(x1,tree.tree2)
            x = self._build_root(tree.root, x2, x1, *children)
        else:
            children.append(x1)
            x = self._build_tree(x1, tree.tree2, children=children)
        return x
        
    def build(self):
        inputs = self.input

        x = self._build_list(inputs, self.base_layer)
        #print("## After base layer",x.shape)
        x = self._build_list(x, self.level0)
        #print("## After lev0 layer",x.shape)
        x = self._build_list(x, self.level1)
        #print("## After lev1 layer",x.shape)
        x = self._build_tree(x, self.level2)
        #print("## After lev2 layer",x.shape)
        x = self._build_tree(x, self.level3)
        #print("## After lev3 layer",x.shape)
        x = self._build_tree(x, self.level4)
        #print("## After lev4 layer",x.shape)
        x = self._build_tree(x, self.level5)
        #print("## After lev5 layer",x.shape)
        #TODO clean the debug comemnted code when done
        return Model(inputs = inputs, outputs=x)

# Testing model creation

In [5]:
m = DlaKeras(activation_function="hswish", debug=True).build()

dummy = np.zeros((1,512,512,3), dtype=float)
dummy_shape=m.predict(dummy).shape
print("Output shape calculated:", dummy_shape)

print(m.summary())

[NODE: L2_ lr:False rdim:128 downs: False proj: False levs: 1]
  R1> conv,bn,act 
  T1> conv,bn
  T2> conv,bn
[NODE: L3_ lr:True rdim:320 downs: False proj: False levs: 2]
  R1> None
  T1> [NODE: L3_1 lr:True rdim:320 downs: False proj: False levs: 1]
      R1> conv,bn,act 
      T1> conv,bn
      T2> conv,bn
  T2> [NODE: L3_2 lr:True rdim:576 downs: True proj: True levs: 1]
      R1> conv,bn,act 
      T1> conv,bn
      T2> conv,bn
[NODE: L4_ lr:True rdim:640 downs: False proj: False levs: 2]
  R1> None
  T1> [NODE: L4_1 lr:True rdim:640 downs: False proj: False levs: 1]
      R1> conv,bn,act 
      T1> conv,bn
      T2> conv,bn
  T2> [NODE: L4_2 lr:True rdim:1152 downs: True proj: True levs: 1]
      R1> conv,bn,act 
      T1> conv,bn
      T2> conv,bn
[NODE: L5_ lr:True rdim:1280 downs: False proj: False levs: 1]
  R1> conv,bn,act 
  T1> conv,bn
  T2> conv,bn
Output shape calculated: (1, 16, 16, 512)
___________________________________________________________________________________

# Dummy train: does it train at all ?

In [6]:
def dummy_train(model:Model,n=50, epochs=3):
    rand = np.random.rand(1,512,512,1)
    dummy_inputs = np.repeat(np.repeat(rand, n, axis=0),3,axis=-1)
    dummy_outputs = np.zeros((n,dummy_shape[1],dummy_shape[2],dummy_shape[3]))
    model.fit(dummy_inputs, dummy_outputs, batch_size=4, epochs=epochs)
    
print("Training:")
m = DlaKeras(activation_function="hswish").build()
optimizer = SGD(lr=0.1)
m.compile(optimizer=optimizer, loss='mse')
dummy_train(m)

Training:
Epoch 1/3
50/50 [==============================] - 80s - loss: 0.2212    
Epoch 2/3
50/50 [==============================] - 81s - loss: 0.1252    
Epoch 3/3
50/50 [==============================] - 81s - loss: 0.0740    
